In [1]:
import pandas as pd

In [2]:
YEAR = 2023

In [3]:
TARGET = 'customer'
churn_df = pd.read_csv(f'../dataset/iMB_{TARGET.lower()}_data_{YEAR}.csv', encoding='euc-kr')

TARGET = 'card'
card_df = pd.read_csv(f'../dataset/imbank_{TARGET.lower()}_data/iMBANK_{TARGET.upper()}_DATA_{YEAR}(K-DigitalTraining).csv', usecols=['거래년월', '고객ID', '가맹점업종명', '승인건수','승인금액'], encoding='euc-kr') 

In [4]:
churn_df= pd.merge(card_df[['거래년월', '고객ID', '가맹점업종명', '승인건수', '승인금액']], churn_df[['기준년월', '고객ID', '연령대', '자택_시도', '분기', '이탈여부']], left_on=['고객ID', '거래년월'], right_on=['고객ID', '기준년월'], how='inner')

In [ ]:
def classify_category(row):
    if row in ["인터넷Mall", "인터넷P/G", "인터넷종합Mall"]:
        return "인터넷 결제"
    elif row in [ "1급 호텔", "2급 호텔", "특급호텔", "콘도", "골프경기장", "골프연습장", "종합레져타운", "헬스클럽", "수 영 장", "스크린골프", "사우나", "안마/스포츠마사지", "카페트, 커튼, 천막, 지물", "레져업소"]:
        return "숙박/레저"
    elif row in ["갈비전문점", "서양음식", "일반한식", "일식·회집", "중국식", "한정식", "스넥", "주점", "단란주점", "유흥주점", "칵테일바"]:
        return "외식"
    elif row in ["고속버스", "택시", "택시회사", "여객선", "철도", "화물운송"]:
        return "교통/운송"
    elif row in ["CATV홈쇼핑", "주유소", "GS주유소", "SK주유소", "E1가스충전소", "쌍용S-OIL가스충전소", "현대정유가스충전소", "농,축협직영매장", "농협하나로클럽", "대형할인점", "편 의 점", "슈퍼마켓", "PG상품권", "상 품 권"]:
        return "소매/판매"
    elif row in ["초중고교육기관", "기능학원", "보습학원", "예·체능계학원", "외국어학원", "학원", "학습지교육"]:
        return "교육"
    elif row in ["병 원(응급실운영)", "동물병원", "치과병원", "치과의원", "의 원", "한 의 원", "한방병원",
                 "약 국", "의료용품", "기타의료기관 및 기타의료기기", "피부미용실", "건강진단", "인삼제품", "홍삼제품"]:
        return "의료/건강"
    elif row in ["생명보험", "손해보험", "기타보험", "비씨카드 정산용(할인)", "비씨카드 정산용가맹점"]:
        return "금융/보험"
    elif row in ["관광여행", "공공요금대행서비스/소득공제대상", "공공요금대행서비스/소득공제비대상",
                 "사무서비스", "사무·통신기기수리", "위탁급식업", "보관및 창고업", "노 래 방", "사무용 OA기기",
                 "종합용역", "보일러·펌프·샷시", "택시회사", "기념품점", "볼 링 장", "기념품점",
                 "전자상거래상품권전문판매", "기타잡화", "가전제품"]:
        return "기타 서비스"
    else:
        return "기타"

In [6]:
churn_df = churn_df[churn_df['승인건수']=='5번이하']
churn_df.drop('기준년월', axis=1, inplace=True)
kind_df = churn_df.groupby(['가맹점업종명', '분기', '연령대'])['고객ID'].count().reset_index()

In [11]:
churn_df

,거래년월,고객ID,가맹점업종명,승인건수,승인금액,연령대,자택_시도,분기,이탈여부
0,202312,3557388137,일반한식,5번이하,20000,20대,경상북도,4,안전
1,202312,3557255233,일반한식,5번이하,10000,20대,경상북도,4,안전
2,202312,3474259363,일반한식,5번이하,20000,20대,경상북도,4,위험
3,202312,3698202806,일반한식,5번이하,30000,20대,경상북도,4,위험
4,202312,7676586511,자동차정비,5번이하,100000,50대,경상북도,4,위험
...,...,...,...,...,...,...,...,...,...
62031428,202301,7210942935,일식·회집,5번이하,50000,50대,경상북도,1,안전
62031429,202301,3857466791,일식·회집,5번이하,20000,50대,경상북도,1,안전
62031430,202301,4075163762,일식·회집,5번이하,40000,10대미만,경상북도,1,안전
62031431,202301,2956984953,일식·회집,5번이하,10000,20대,경상북도,1,안전


In [8]:
kind_df.columns = ['가맹점업종명', '분기', '연령대', '거래수']
kind_df['가맹점업종분류']= churn_df['가맹점업종명'].apply(classify_category)

In [9]:
kind_df

,가맹점업종명,분기,연령대,거래수,가맹점업종분류
0,1급 호텔,1,10대미만,6,외식
1,1급 호텔,1,20대,123,외식
2,1급 호텔,1,30대,197,외식
3,1급 호텔,1,40대,374,외식
4,1급 호텔,1,50대,567,기타
...,...,...,...,...,...
4762,화장품,4,20대,2831,소매/판매
4763,화장품,4,30대,2653,소매/판매
4764,화장품,4,40대,4239,소매/판매
4765,화장품,4,50대,9398,소매/판매


In [ ]:
kind_df.to_csv(f'../dataset/industry/industry_{YEAR}.csv', encoding = 'euc-kr', index=False)

In [ ]:
YEAR = 2022
kind_df = pd.read_csv(f'../dataset/industry/industry_{YEAR}.csv', encoding = 'euc-kr')
# 분기 삭제

In [26]:
df=kind_df[kind_df['가맹점업종분류'].isin(['숙박/레저', '교육', '기타 서비스', '교통/운송'])]

In [28]:
df[df['연령대']=='30대']

,가맹점업종명,분기,연령대,거래수,가맹점업종분류
79,DP&E,1,30대,1251,기타 서비스
1675,문구용품,4,30대,8577,교통/운송
1693,문화취미기타,3,30대,309,교통/운송
1705,미곡상,1,30대,324,교통/운송
1711,미곡상,2,30대,313,교통/운송
1717,미곡상,3,30대,309,교통/운송
1723,미곡상,4,30대,337,교통/운송
1729,미용원,1,30대,18242,교통/운송
1735,미용원,2,30대,20075,교통/운송
1741,미용원,3,30대,19194,교통/운송


In [15]:
# 필요업종: '숙박/레저', '교육', '기타 서비스', '교통/운송'